# CounterNet

> A prediction-aware recourse model

* Paper link: https://arxiv.org/abs/2109.07557

In [1]:
#| default_exp methods.counternet

In [2]:
#| include: false
%load_ext autoreload
%autoreload 2
from ipynb_path import *

In [3]:
#| export
from __future__ import annotations
from relax.import_essentials import *
from relax.methods.base import CFModule, ParametricCFModule
from relax.base import BaseConfig
from relax.utils import auto_reshaping, grad_update, validate_configs
from relax.data_utils import Feature, FeaturesList
from relax.ml_model import MLP, MLPBlock
from relax.data_module import DataModule

Using JAX backend.


## CounterNet Model


In [4]:
#| exporti
class CounterNetConfig(BaseConfig):
    """Configurator of `CounterNetModel`."""

    enc_sizes: List[int] = Field([50,10], description='Encoder sizes.')
    pred_sizes: List[int] = Field([10], description='Predictor sizes.')
    exp_sizes: List[int] = Field([50, 50], description='CF generator sizes.')
    dropout_rate: float = Field(0.3, description='Dropout rate.')
    lr: float = 0.003
    lambda_1: float = 1.0
    lambda_2: float = 0.2
    lambda_3: float = 0.1

In [6]:
#| export
class CounterNetModel(keras.Model):
    """CounterNet Model"""
    def __init__(self, config: Dict | CounterNetConfig = None):
        """CounterNet model architecture."""
        super().__init__()
        config = CounterNetConfig() if config is None else config
        self.config: CounterNetConfig = validate_configs(config, CounterNetConfig)
        self.dropout_rate = self.config.dropout_rate

    def build(self, input_shape):
        self.encoder = keras.Sequential([
            MLPBlock(size, self.dropout_rate) for size in self.config.enc_sizes
        ])
        # predictor
        self.predictor = keras.Sequential([
            MLPBlock(size, self.dropout_rate) for size in self.config.pred_sizes
        ])
        self.pred_linear = keras.layers.Dense(2, activation='softmax')
        # explainer
        self.explainer = MLP(
            self.config.exp_sizes, output_size=input_shape[-1], 
            dropout_rate=self.dropout_rate, last_activation='linear'
        )

    def call(self, x, training=False):
        z = self.encoder(x, training=training)
        # predict y_hat
        pred = self.predictor(z, training=training)
        y_hat = self.pred_linear(pred)
        # explain x
        z_exp = keras.ops.concatenate([z, pred], axis=-1)
        cfs = self.explainer(z_exp, training=training)
        return pred, cfs
    
    def stateless_forward(self, params, xs, training=False):
        (pred, cfs), non_trainable_params = self.stateless_call(
            *params, xs, training=training
        )
        (cf_ys, _), non_trainable_params = self.stateless_call(
            *params, cfs, training=training
        )
        return pred, cfs, cf_ys
    
    def pred_loss_fn(self, params, xs, ys, training=False):
        (pred, cfs), non_trainable_params = self.stateless_call(
            *params, xs, training=training
        )
        pred_loss = self.config.lambda_1 * self.loss_1(ys, pred)
        return pred_loss
    
    def exp_loss_fn(self, params, xs, ys, training=False):
        pred, cfs, cf_ys = self.forward(params, xs, training=training)
        # TODO: Do we need the hard or soft labels?
        y_prime = 1 - jnp.round(pred)
        loss_2 = self.loss_2(y_prime, cf_ys)
        loss_3 = self.loss_3(xs, cfs)
        exp_loss = self.config.lambda_2 * loss_2 + self.config.lambda_3 * loss_3
        return exp_loss
    
    def freeze_predictor(self, freeze=True):
        freezing_layers = [self.encoder, self.predictor, self.pred_linear]
        for layer in freezing_layers:
            layer.trainable = not freeze

In [6]:
def pred_step(params, batch, opt_state, model):
    opt = model.opt_1
    loss, grads = jax.value_and_grad(model.pred_loss_fn)(params, batch, training=True)
    trainable_params, opt_state = opt.stateless_apply(
        opt_state, grads, trainable_params
    )
    return (trainable_params, opt_state), loss


def exp_step(params, batch, opt_state, model):
    opt = model.opt_2
    loss, grads = jax.value_and_grad(model.exp_loss_fn)(params, batch, training=True)
    trainable_params, opt_state = opt.stateless_apply(
        opt_state, grads, trainable_params
    )
    return (trainable_params, opt_state), loss

def train_step(
    state, 
    batch,
    model: CounterNetModel,
):
    trainable_params, non_trainable_params, (opt_1_state, opt_2_state), metrics_vars = state
    xs, ys = batch

    # Stage 1: Train predictor
    model.freeze_predictor(freeze=False)
    (trainable_params, opt_1_state), loss = pred_step(
        (trainable_params, non_trainable_params), (xs, ys), opt_1_state, model
    )

    # Stage 2: Train explainer
    model.freeze_predictor(freeze=True)
    (trainable_params, opt_2_state), loss = exp_step(
        (trainable_params, non_trainable_params), (xs, ys), opt_2_state, model
    )

    # Update state
    state = trainable_params, non_trainable_params, (opt_1_state, opt_2_state), metrics_vars
    return state, loss

In [7]:
cfnet = CounterNetModel()
opt_1 = keras.optimizers.Adam(learning_rate=0.01)

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [8]:
cfnet(jnp.zeros((10, 2)))

(Array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32),
 Array([[0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.]], dtype=float32))

In [9]:
cfnet.variables

[<KerasVariable shape=(2, 50), dtype=float32, name=variable>,
 <KerasVariable shape=(50,), dtype=float32, name=variable_1>,
 <KerasVariable shape=(2,), dtype=uint32, name=seed_generator_state>,
 <KerasVariable shape=(50, 10), dtype=float32, name=variable_2>,
 <KerasVariable shape=(10,), dtype=float32, name=variable_3>,
 <KerasVariable shape=(2,), dtype=uint32, name=seed_generator_state>,
 <KerasVariable shape=(10, 10), dtype=float32, name=variable_4>,
 <KerasVariable shape=(10,), dtype=float32, name=variable_5>,
 <KerasVariable shape=(2,), dtype=uint32, name=seed_generator_state>,
 <KerasVariable shape=(10, 2), dtype=float32, name=variable_6>,
 <KerasVariable shape=(2,), dtype=float32, name=variable_7>,
 <KerasVariable shape=(20, 50), dtype=float32, name=variable_8>,
 <KerasVariable shape=(50,), dtype=float32, name=variable_9>,
 <KerasVariable shape=(2,), dtype=uint32, name=seed_generator_state>,
 <KerasVariable shape=(50, 50), dtype=float32, name=variable_10>,
 <KerasVariable shape=(5

In [10]:
cfnet.freeze_predictor(freeze=True)
cfnet.trainable_variables

[<KerasVariable shape=(20, 50), dtype=float32, name=variable_8>,
 <KerasVariable shape=(50,), dtype=float32, name=variable_9>,
 <KerasVariable shape=(50, 50), dtype=float32, name=variable_10>,
 <KerasVariable shape=(50,), dtype=float32, name=variable_11>,
 <KerasVariable shape=(50, 2), dtype=float32, name=variable_12>,
 <KerasVariable shape=(2,), dtype=float32, name=variable_13>]

In [25]:
cfnet.non_trainable_variables

[<KerasVariable shape=(2, 50), dtype=float32, name=variable_14>,
 <KerasVariable shape=(50,), dtype=float32, name=variable_15>,
 <KerasVariable shape=(2,), dtype=uint32, name=seed_generator_state>,
 <KerasVariable shape=(50, 10), dtype=float32, name=variable_16>,
 <KerasVariable shape=(10,), dtype=float32, name=variable_17>,
 <KerasVariable shape=(2,), dtype=uint32, name=seed_generator_state>,
 <KerasVariable shape=(10, 10), dtype=float32, name=variable_18>,
 <KerasVariable shape=(10,), dtype=float32, name=variable_19>,
 <KerasVariable shape=(2,), dtype=uint32, name=seed_generator_state>,
 <KerasVariable shape=(10, 2), dtype=float32, name=variable_20>,
 <KerasVariable shape=(2,), dtype=float32, name=variable_21>,
 <KerasVariable shape=(2,), dtype=uint32, name=seed_generator_state>,
 <KerasVariable shape=(2,), dtype=uint32, name=seed_generator_state>]

In [21]:
opt_1.build(cfnet.trainable_variables)

## CounterNet Training Module

Define the `CounterNetTrainingModule` for training `CounterNetModel`.

In [ ]:
#| export 
def partition_trainable_params(params: hk.Params, trainable_name: str):
    trainable_params, non_trainable_params = hk.data_structures.partition(
        lambda m, n, p: trainable_name in m, params
    )
    return trainable_params, non_trainable_params


In [ ]:
#| export
def project_immutable_features(x, cf: jax.Array, imutable_idx_list: List[int]):
    cf = cf.at[:, imutable_idx_list].set(x[:, imutable_idx_list])
    return cf

In [ ]:
#| export
class CounterNetTrainingModuleConfigs(BaseParser):
    lr: float = 0.003
    lambda_1: float = 1.0
    lambda_2: float = 0.2
    lambda_3: float = 0.1


In [ ]:
#| export
class CounterNetTrainingModule(BaseTrainingModule):
    _data_module: TabularDataModule

    def __init__(self, m_configs: Dict[str, Any]):
        self.save_hyperparameters(m_configs)
        self.net = make_model(m_configs, CounterNetModel)
        self.configs = validate_configs(m_configs, CounterNetTrainingModuleConfigs)
        # self.configs = CounterNetTrainingModuleConfigs(**m_configs)
        self.opt_1 = optax.adam(learning_rate=self.configs.lr)
        self.opt_2 = optax.adam(learning_rate=self.configs.lr)

    def init_net_opt(self, data_module: TabularDataModule, key):
        # hook data_module
        self._data_module = data_module
        X, _ = data_module.train_dataset[:]

        # manually init multiple opts
        params, opt_1_state = init_net_opt(
            self.net, self.opt_1, X=X[:100], key=key
        )
        trainable_params, _ = partition_trainable_params(
            params, trainable_name="counter_net_model/Explainer"
        )
        opt_2_state = self.opt_2.init(trainable_params)
        return params, (opt_1_state, opt_2_state)

    @partial(jax.jit, static_argnames=["self", "is_training"])
    def forward(self, params, rng_key, x, is_training: bool = True):
        # first forward to get y_pred and normalized cf
        y_pred, cf = self.net.apply(params, rng_key, x, is_training=is_training)
        cf = self._data_module.apply_constraints(x, cf, hard=not is_training)

        # second forward to calulate cf_y
        cf_y, _ = self.net.apply(params, rng_key, cf, is_training=is_training)
        return y_pred, cf, cf_y

    @partial(jax.jit, static_argnames=["self"])
    def predict(self, params, rng_key, x):
        y_pred, _ = self.net.apply(params, rng_key, x, is_training=False)
        return y_pred

    def generate_cfs(self, X: Array, params, rng_key) -> chex.ArrayBatched:
        y_pred, cfs = self.net.apply(params, rng_key, X, is_training=False)
        # cfs = cfs + X
        cfs = self._data_module.apply_constraints(X, cfs, hard=True)
        return cfs

    @partial(jax.jit, static_argnames=["self"])
    def loss_fn_1(self, y_pred, y):
        return jnp.mean(vmap(optax.l2_loss)(y_pred, y))

    @partial(jax.jit, static_argnames=["self"])
    def loss_fn_2(self, cf_y, y_prime):
        return jnp.mean(vmap(optax.l2_loss)(cf_y, y_prime))

    @partial(jax.jit, static_argnames=["self"])
    def loss_fn_3(self, x, cf):
        return jnp.mean(vmap(optax.l2_loss)(x, cf))

    @partial(jax.jit, static_argnames=["self", "is_training"])
    def pred_loss_fn(self, params, rng_key, batch, is_training: bool = True):
        x, y = batch
        y_pred, cf = self.net.apply(params, rng_key, x, is_training=is_training)
        return self.configs.lambda_1 * self.loss_fn_1(y_pred, y)

    @partial(jax.jit, static_argnames=["self", "is_training"])
    def exp_loss_fn(
        self,
        trainable_params,
        non_trainable_params,
        rng_key,
        batch,
        is_training: bool = True,
    ):
        # merge trainable and non_trainable params
        params = hk.data_structures.merge(trainable_params, non_trainable_params)
        x, y = batch
        y_pred, cf, cf_y = self.forward(params, rng_key, x, is_training=is_training)
        y_prime = 1 - jnp.round(y_pred)
        loss_2, loss_3 = self.loss_fn_2(cf_y, y_prime), self.loss_fn_3(x, cf)
        return self.configs.lambda_2 * loss_2 + self.configs.lambda_3 * loss_3

    @partial(jax.jit, static_argnames=["self",])
    def _predictor_step(self, params, opt_state, rng_key, batch):
        grads = jax.grad(self.pred_loss_fn)(params, rng_key, batch)
        upt_params, opt_state = grad_update(grads, params, opt_state, self.opt_1)
        return upt_params, opt_state

    @partial(jax.jit, static_argnames=["self",])
    def _explainer_step(self, params, opt_state, rng_key, batch):
        trainable_params, non_trainable_params = partition_trainable_params(
            params, trainable_name="counter_net_model/Explainer"
        )
        grads = jax.grad(self.exp_loss_fn)(
            trainable_params, non_trainable_params, rng_key, batch
        )
        upt_trainable_params, opt_state = grad_update(
            grads, trainable_params, opt_state, self.opt_2
        )
        upt_params = hk.data_structures.merge(
            upt_trainable_params, non_trainable_params
        )
        return upt_params, opt_state

    @partial(jax.jit, static_argnames=["self"])
    def _training_step(
        self,
        params: hk.Params,
        opts_state: Tuple[optax.GradientTransformation, optax.GradientTransformation],
        rng_key: random.PRNGKey,
        batch: Tuple[jnp.array, jnp.array],
    ):
        opt_1_state, opt_2_state = opts_state
        params, opt_1_state = self._predictor_step(params, opt_1_state, rng_key, batch)
        upt_params, opt_2_state = self._explainer_step(
            params, opt_2_state, rng_key, batch
        )
        return upt_params, (opt_1_state, opt_2_state)

    def _training_step_logs(self, params, rng_key, batch):
        x, y = batch
        y_pred, cf, cf_y = self.forward(params, rng_key, x, is_training=False)
        y_prime = 1 - jnp.round(y_pred)

        loss_1, loss_2, loss_3 = (
            self.loss_fn_1(y_pred, y),
            self.loss_fn_2(cf_y, y_prime),
            self.loss_fn_3(x, cf),
        )
        logs = {
            "train/train_loss_1": loss_1.item(),
            "train/train_loss_2": loss_2.item(),
            "train/train_loss_3": loss_3.item(),
        }
        return logs

    def training_step(
        self,
        params: hk.Params,
        opts_state: Tuple[optax.OptState, optax.OptState],
        rng_key: random.PRNGKey,
        batch: Tuple[jnp.array, jnp.array],
    ) -> Tuple[hk.Params, Tuple[optax.OptState, optax.OptState]]:
        upt_params, (opt_1_state, opt_2_state) = self._training_step(
            params, opts_state, rng_key, batch
        )

        logs = self._training_step_logs(upt_params, rng_key, batch)
        self.log_dict(logs)
        return upt_params, (opt_1_state, opt_2_state)

    def validation_step(self, params, rng_key, batch):
        x, y = batch
        y_pred, cf, cf_y = self.forward(params, rng_key, x, is_training=False)
        y_prime = 1 - jnp.round(y_pred)

        loss_1, loss_2, loss_3 = (
            self.loss_fn_1(y_pred, y),
            self.loss_fn_2(cf_y, y_prime),
            self.loss_fn_3(x, cf),
        )
        loss_1, loss_2, loss_3 = map(np.asarray, (loss_1, loss_2, loss_3))
        logs = {
            "val/accuracy": accuracy(y, y_pred),
            "val/validity": accuracy(cf_y, y_prime),
            "val/proximity": proximity(x, cf),
            "val/val_loss_1": loss_1,
            "val/val_loss_2": loss_2,
            "val/val_loss_3": loss_3,
            "val/val_loss": loss_1 + loss_2 + loss_3,
        }
        self.log_dict(logs)
        return logs

## CounterNet Explanation Module


![CounterNet architecture](./images/CounterNet-architecture.svg)

`CounterNet` consists of three objectives:

1. **predictive accuracy**: the predictor network should output accurate predictions $\hat{y}_x$; 
2. **counterfactual validity**: CF examples $x'$ produced by the CF generator network should be valid (e.g. $\hat{y}_{x} + \hat{y}_{x'}=1$);
3. **minimizing cost of change**: minimal modifications should be required to change input instance $x$ to CF example $x'$.

The objective function of `CounterNet`:

$$
\operatorname*{argmin}_{\mathbf{\theta}} \frac{1}{N}\sum\nolimits_{i=1}^{N} 
    \bigg[ 
    \lambda_1 \cdot \! \underbrace{\left(y_i- \hat{y}_{x_i}\right)^2}_{\text{Prediction Loss}\ (\mathcal{L}_1)} + 
    \;\lambda_2 \cdot \;\; \underbrace{\left(\hat{y}_{x_i}- \left(1 - \hat{y}_{x_i'}\right)\right)^2}_{\text{Validity Loss}\ (\mathcal{L}_2)} \,+ 
    \;\lambda_3 \cdot \!\! \underbrace{\left(x_i- x'_i\right)^2}_{\text{Cost of change Loss}\ (\mathcal{L}_3)}
    \bigg]
$$

`CounterNet` applies two-stage gradient updates to `CounterNetModel` 
for each `training_step` (see `CounterNetTrainingModule`).

1. The first gradient update optimizes for predictive accuracy: 
$\theta^{(1)} = \theta^{(0)} - \nabla_{\theta^{(0)}} (\lambda_1 \cdot \mathcal{L}_1)$.
2. The second gradient update optimizes for generating CF explanation:
$\theta^{(2)}_g = \theta^{(1)}_g - \nabla_{\theta^{(1)}_g} (\mathcal \lambda_2 \cdot \mathcal{L}_2 + \lambda_3 \cdot \mathcal{L}_3)$

The design choice of this optimizing procedure is made due to *improved convergence of the model*,
and *improved adversarial robustness of the predictor network*. 
The [CounterNet paper](https://arxiv.org/abs/2109.07557) elaborates the design choices.


In [ ]:
#| export
class CounterNetConfigs(CounterNetTrainingModuleConfigs, CounterNetModelConfigs):
    """Configurator of `CounterNet`."""

    enc_sizes: List[int] = Field(
        [50,10], description="Sequence of layer sizes for encoder network."
    )
    dec_sizes: List[int] = Field(
        [10], description="Sequence of layer sizes for predictor."
    ) 
    exp_sizes: List[int] = Field(
        [50, 50], description="Sequence of layer sizes for CF generator."
    )
    
    dropout_rate: float = Field(
        0.3, description="Dropout rate."
    )
    lr: float = Field(
        0.003, description="Learning rate for training `CounterNet`."
    ) 
    lambda_1: float = Field(
        1.0, description=" $\lambda_1$ for balancing the prediction loss $\mathcal{L}_1$."
    ) 
    lambda_2: float = Field(
        0.2, description=" $\lambda_2$ for balancing the prediction loss $\mathcal{L}_2$."
    ) 
    lambda_3: float = Field(
        0.1, description=" $\lambda_3$ for balancing the prediction loss $\mathcal{L}_3$."
    )


In [ ]:
#| export
class CounterNet(BaseCFModule, BaseParametricCFModule, BasePredFnCFModule):
    """API for CounterNet Explanation Module."""
    params: hk.Params = None
    module: CounterNetTrainingModule
    name: str = 'CounterNet'

    def __init__(
        self, 
        m_configs: dict | CounterNetConfigs = None # configurator of hyperparamters; see `CounterNetConfigs`
    ):
        if m_configs is None:
            m_configs = CounterNetConfigs()
        self.module = CounterNetTrainingModule(m_configs)

    def _is_module_trained(self):
        return not (self.params is None)
    
    def train(
        self, 
        datamodule: TabularDataModule, # data module
        t_configs: TrainingConfigs | dict = None, # training configs
        *args, **kwargs
    ):
        _default_t_configs = dict(
            n_epochs=100, batch_size=128
        )
        if t_configs is None: t_configs = _default_t_configs
        params, _ = train_model(self.module, datamodule, t_configs)
        self.params = params

    @auto_reshaping('x')
    def generate_cf(self, x: jnp.ndarray, pred_fn = None) -> jnp.ndarray:
        return self.module.generate_cfs(x, self.params, rng_key=jax.random.PRNGKey(0))

    def generate_cfs(self, X: jnp.ndarray, pred_fn = None) -> jnp.ndarray:
        return self.module.generate_cfs(X, self.params, rng_key=jax.random.PRNGKey(0))
    
    def pred_fn(self, X: jax.Array):
        rng_key = jax.random.PRNGKey(0)
        y_pred = self.module.predict(self.params, rng_key, X)
        return y_pred


#### Basic usage of `CounterNet`

Prepare data:

In [ ]:
from relax.data import load_data

In [ ]:

dm = load_data("adult", data_configs=dict(sample_frac=0.1))

/Users/chuck/opt/anaconda3/envs/relax/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Define `CounterNet`:

In [ ]:
counternet = CounterNet()

In [ ]:
assert isinstance(counternet, BaseParametricCFModule)
assert isinstance(counternet, BaseCFModule)
assert isinstance(counternet, BasePredFnCFModule)
assert hasattr(counternet, 'pred_fn')

Train the model:

In [ ]:
#| output: false
t_configs = dict(n_epochs=1, batch_size=128)
counternet.train(dm, t_configs=t_configs)

/Users/chuck/opt/anaconda3/envs/relax/lib/python3.8/site-packages/relax/_ckpt_manager.py:47: UserWarning: `monitor_metrics` is not specified in `CheckpointManager`. No checkpoints will be stored.
  warnings.warn(
Epoch 0: 100%|██████████| 20/20 [00:05<00:00,  3.62batch/s, train/train_loss_1=0.0568, train/train_loss_2=0.221, train/train_loss_3=0.117]


Predict labels

In [ ]:
X, y = dm.test_dataset[:]
y_pred = counternet.pred_fn(X)
assert y_pred.shape == (len(y), 1)

Generate a CF explanation for a given `x`.

In [ ]:
x, _ = dm.test_dataset[0]
cf = counternet.generate_cf(x)
assert x.shape == cf.shape
assert cf.shape == (29,)

Generate CF explanations for given `x`.

In [ ]:
X, _ = dm.test_dataset[:]
cfs = counternet.generate_cfs(X)
assert X.shape == cfs.shape